# Movie Project
- Clint Atterberry

# Business Problem
*For this project, you have been hired to produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. Ultimately, you will use this database to analyze what makes a movie successful, and will provide recommendations to the stakeholder on how to make a successful movie.*

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [3]:
# load login keys
with open('/Users/oneda/.secret/movie_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# login with API-Key
tmdb.API_KEY =  login['api-key']

In [5]:
# Load previous data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'Basics_Data.tsv',
 'Ratings_Data.csv',
 'titles_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz']

# Load JSON File

In [6]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0094859,movie,Chief Zabu,Chief Zabu,0,2016,NaN,74,Comedy


In [7]:
def write_json(new_data, filename):
    # referenced from https://youtu.be/T8kweKdrhSI?t=1503
    
    with open(filename, 'r+') as file:
        # load data into dict
        file_data = json.load(file)
        ## append or extend data to file
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # create offset from data
        file.seek(0)
        # convert back to json
        json.demp(file_data, file)

# Setup Variables

In [8]:
# Define the years
YEARS_TO_GET =[2000,2001]

# test with movie_id = 603
movie_id = 603

# make movie object using .Movies
movie = tmdb.Movies(movie_id)

# movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/n2nm4aZRmXyJ9LT4xQX9X6ThcP7.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 84.898,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [10]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    # Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # Saving movie ids to list
    movie_ids = df['tconst'].copy() # to_list()
    movie_ids
    
    # Load exisiting data from json into a df called 'previous_df'
    previous_df = pd.read_json(JSON_FILE)
    previous_df

    # Filter out any ids that are already in the JSON_File
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    # Get index and movie id from list
    # INNER Loop

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position=1,
                                 leave=True):
        # try to retrieve data
        try:
            # make movie object using .Movies
            movie = tmdb.Movies(movie_id)
            # save the .info .releases dictionaries
            info = movie.info()
            releases = movie.releases()
            # Loop through countries in releases
            for c in releases['countries']:
                # if the country abbreviation==US
                if c['iso_3166_1' ] =='US':
                    # save a key in the info dict with the new info
                    info['budget'] = c['budget']
                    info['revenue'] = c['revenue']
                    info['certification'] = c['certification']

            # Append/Extend results to existing file
            write_json(info['budget'], JSON_FILE)
            write_json(info['revenue'], JSON_FILE)
            write_json(info['certification'], JSON_FILE)

            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails, make a dixt with just the id and None for certification
        except Exception as e:
            continue


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1408 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1528 [00:00<?, ?it/s]

AttributeError: 'int' object has no attribute 'keys'

In [12]:
year_df = pd.read_json(JSON_FILE)
year_df.to_csv(f"{FOLDER}tmdb_data_{YEAR}.csv.gz", compression='gzip', index=False)

ValueError: Expected object or value

In [ ]:
basics = pd.read_csv("Data/final_tmdb_data_2000.csv.gz", low_memory = False)
basics.info()

In [ ]:
basics.columns